# Phase 1: Extracting Triplets from Informal Proofs

Add src to the Python Path in the Notebook

In [1]:
%load_ext autoreload
%autoreload 2
import sys
import os


# Add the project root directory to the Python path
sys.path.append(os.path.abspath(os.path.join("../..")))

## 1. Read Informal Proves
Read the LaTeX file. This file contains the informal proofs of the theorems in the book.

In [2]:
from IPython.display import display, Math, Latex
import re
from src.utils.file_utils import read_proof

# Load LaTeX proof
proof_latex = read_proof("../data/proofs/english/problem2/proof1.tex")

# Find the start and end positions
start = proof_latex.find(r"\begin{document}") + len(r"\begin{document}")
end = proof_latex.find(r"\end{document}")

# Extract the content between \begin{document} and \end{document}
informal_proof = proof_latex[start:end].strip()


# Replace any \section{...} with ## ...
informal_proof = re.sub(r"\\section\{([^}]+)\}", r"## \1", informal_proof)

# Display the LaTeX content
display(Latex(informal_proof))

<IPython.core.display.Latex object>

## 2. Extract Triplet proofs from Informal Proofs

In [3]:
from src.phase1.extract_triplets import extract_triplets

# Extract triplets
triplet = extract_triplets(informal_proof)
triplet

Triplet(entities=[Entity(id='A', name='A', label='Matrix A', type='Matrix'), Entity(id='B', name='B', label='Matrix B', type='Matrix'), Entity(id='c_{ij}', name='c_{ij}', label='Element of product matrix', type='Element'), Entity(id='a_{ij}', name='a_{ij}', label='Element of matrix A', type='Element'), Entity(id='b_{ij}', name='b_{ij}', label='Element of matrix B', type='Element'), Entity(id='Theorem 2.5(i)', name='Theorem 2.5(i)', label='Theorem 2.5(i)', type='Theorem')], relations=[Relation(source='A', target='B', type='proves', name='Matrix multiplication of upper triangular matrices results in upper triangular matrix'), Relation(source='c_{ij}', target='a_{ik}', type='defines', name='Element of product matrix defined by elements of A and B'), Relation(source='c_{ij}', target='b_{kj}', type='defines', name='Element of product matrix defined by elements of A and B'), Relation(source='c_{ii}', target='a_{ii}', type='defines', name='Diagonal element of product matrix defined by diagona

## 3. Store Triplets into Neo4J DB

In [4]:
from src.utils.neo4j_utils import Neo4JUtils

# Initialize Neo4JUtils
neo4j = Neo4JUtils("bolt://localhost:7687", ("neo4j", "password"))

# Clean the database (delete all nodes and relationships)
neo4j.clean_database()

# Add nodes and relationships with step tracking
for entity in triplet.entities:
    neo4j.create_node(entity)  # Uses the current step (default is 0)
for relation in triplet.relations:
    neo4j.create_relation(relation)  # Uses the current step (default is 0)

# Increment the step counter for the next set of changes
neo4j.increment_step()

# Clean the database (delete nodes and relationships with step > 1)
neo4j.clean_database(step=1)

# Add or modify nodes and relationships in the next step
# Example:
# neo4j.create_node(new_entity)  # This will use the updated step counter (1)
# neo4j.create_relation(new_relation)  # This will use the updated step counter (1)

# Close the connection
neo4j.close()